In [7]:
pip install nltk

In [11]:
import nltk
import pandas as pd

In [9]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ALPHONSA\AppData\Roaming\nltk_data...


True

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sent_analyzer = SentimentIntensityAnalyzer()

# Example
sentence = "Python is a great programming langauge!"
print(sent_analyzer.polarity_scores(sentence))

{'neg': 0.0, 'neu': 0.477, 'pos': 0.523, 'compound': 0.6588}


In [23]:
# Read the data set
data_url = "https://raw.githubusercontent.com/keitazoumana/VADER_sentiment-Analysis/main/data/testdata.manual.2009.06.14.csv"
sentiment_data = pd.read_csv(data_url)

sentiment_data.head(3)

,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right."
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...


In [24]:
sentiment_data.shape

(497, 6)

In [25]:
def format_data(data):

    last_col = str(data.columns[-1])
    first_col = str(data.columns[0])

    data.rename(columns = {last_col: 'tweet_text', first_col: 'polarity'}, inplace=True) 

    # Change 0, 2, 4 to negative, neutral and positive
    labels = {0: 'negative', 2: 'neutral', 4: 'positive'}
    data['polarity'] = data['polarity'].map(labels)

    # Get only the two columns
    return data[['tweet_text', 'polarity']]

In [26]:
data = format_data(sentiment_data)
data.head(3)

,tweet_text,polarity
0,Reading my kindle2... Love it... Lee childs i...,positive
1,"Ok, first assesment of the #kindle2 ...it fuck...",positive
2,@kenburbary You'll love your Kindle2. I've had...,positive


In [27]:
def format_output(output_dict):
    
    polarity = "neutral"

    if(output_dict['compound']>= 0.05):
        polarity = "positive"

    elif(output_dict['compound']<= -0.05):
        polarity = "negative"

    return polarity

def predict_sentiment(text):
  
    output_dict =  sent_analyzer.polarity_scores(text)
    return format_output(output_dict)

In [28]:
data["vader_prediction"] = data["tweet_text"].apply(predict_sentiment)

C:\Users\ALPHONSA\AppData\Local\Temp\ipykernel_21364\3066860294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["vader_prediction"] = data["tweet_text"].apply(predict_sentiment)


In [29]:
data.sample(5)

,tweet_text,polarity,vader_prediction
331,@foxnews Pelosi should stay in China and never...,negative,neutral
330,I just realized we three monkeys in the white ...,negative,neutral
318,Learning about lambda calculus :),positive,positive
238,LOVING my new Kindle2. Named her Kendra in ca...,positive,positive
321,@ atebits I just finished watching your Stanfo...,positive,positive


In [31]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(data['polarity'], data['vader_prediction'])

print("Accuracy: {}\n".format(accuracy))

# Show the classification report
print(classification_report(data['polarity'], data['vader_prediction']))

Accuracy: 0.716297786720322

              precision    recall  f1-score   support

    negative       0.84      0.64      0.72       177
     neutral       0.67      0.70      0.68       139
    positive       0.67      0.81      0.73       181

    accuracy                           0.72       497
   macro avg       0.73      0.71      0.71       497
weighted avg       0.73      0.72      0.72       497

